In [6]:
%cleanup -f

In [7]:
%%configure -f
{"jars":["/user/livy/repl_jars/dl4j-assembly-0.6.0.jar"],"driverMemory":"3g","executorMemory":"2g","conf":{"spark.driver.extraClassPath":"/home/livy/dl4j-assembly-0.6.0.jar","spark.serializer":"org.apache.spark.serializer.KryoSerializer","spark.kryo.registrator":"org.nd4j.Nd4jRegistrator"}}

In [8]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1477665064581_0005,spark,idle,Link,Link,✔


SparkContext available as 'sc'.
SqlContext available as 'sqlContext'.
res2: org.apache.spark.SparkContext = org.apache.spark.SparkContext@195b735f

In [9]:
import java.io.{ ByteArrayInputStream, ByteArrayOutputStream, File }
import java.util.UUID

import org.apache.hadoop.io.{ BytesWritable, Text }
import org.apache.spark.rdd.RDD
import org.apache.spark.{ SparkConf, SparkContext }
import org.deeplearning4j.datasets.iterator.impl.MnistDataSetIterator
import org.deeplearning4j.nn.api.OptimizationAlgorithm
import org.deeplearning4j.nn.conf.layers.{ DenseLayer, OutputLayer }
import org.deeplearning4j.nn.conf.{ NeuralNetConfiguration, Updater }
import org.deeplearning4j.nn.weights.WeightInit
import org.deeplearning4j.spark.api.{ Repartition, RepartitionStrategy }
import org.deeplearning4j.spark.impl.multilayer.SparkDl4jMultiLayer
import org.deeplearning4j.spark.impl.paramavg.ParameterAveragingTrainingMaster
import org.deeplearning4j.spark.stats.StatsUtils
import org.nd4j.linalg.dataset.DataSet
import org.nd4j.linalg.lossfunctions.LossFunctions

import org.nd4j.linalg.lossfunctions.LossFunctions

In [10]:
val batchSizePerWorker = 32
  val seed = 12345

  import collection.convert.decorateAsScala._

  val iter = new MnistDataSetIterator(batchSizePerWorker, true, seed)
  val data = iter.asScala.toList

  val rdd: RDD[DataSet] = sc.parallelize[DataSet](data)

  val forSequenceFile = rdd.map(dataset => {
    val baos = new ByteArrayOutputStream()
    dataset.save(baos)
    val bytes = baos.toByteArray
    (new Text(UUID.randomUUID().toString), new BytesWritable(bytes))
  })
  forSequenceFile.saveAsSequenceFile("./MnistMLPPreprocessed", None)

  val sequenceFile = sc.sequenceFile("./MnistMLPPreprocessed", classOf[Text], classOf[BytesWritable])
  val trainData = sequenceFile.map(pair => {
    val ds = new DataSet()
    val bais = new ByteArrayInputStream(pair._2.getBytes)
    ds.load(bais)
    ds
  })
  trainData.cache()

res10: trainData.type = MapPartitionsRDD[3] at map at <console>:48

In [11]:
val nnconf = new NeuralNetConfiguration.Builder().
     seed(12345).
     optimizationAlgo(OptimizationAlgorithm.STOCHASTIC_GRADIENT_DESCENT).
     iterations(1).
     activation("relu").
     weightInit(WeightInit.XAVIER).
     learningRate(0.0069).
     updater(Updater.NESTEROVS).momentum(0.9).
     regularization(true).l2(1e-4).
     list().
     layer(0, new DenseLayer.Builder().nIn(28 * 28).nOut(500).build()).
     layer(1, new DenseLayer.Builder().nIn(500).nOut(100).build()).
     layer(2, new OutputLayer.Builder(LossFunctions.LossFunction.NEGATIVELOGLIKELIHOOD).
     activation("softmax").nIn(100).nOut(10).build()).
     pretrain(false).backprop(true).
     build()

nnconf: org.deeplearning4j.nn.conf.MultiLayerConfiguration = 
{
  "backprop" : true,
  "backpropType" : "Standard",
  "confs" : [ {
    "extraArgs" : [ 0 ],
    "l1ByParam" : { },
    "l2ByParam" : { },
    "layer" : {
      "dense" : {
        "activationFunction" : "relu",
        "adamMeanDecay" : "NaN",
        "adamVarDecay" : "NaN",
        "biasInit" : 0.0,
        "biasL1" : 0.0,
        "biasL2" : 0.0,
        "biasLearningRate" : 0.0069,
        "dist" : null,
        "dropOut" : 0.0,
        "epsilon" : "NaN",
        "gradientNormalization" : "None",
        "gradientNormalizationThreshold" : 1.0,
        "l1" : 0.0,
        "l2" : 1.0E-4,
        "layerName" : null,
        "learningRate" : 0.0069,
        "learningRateSchedule" : null,
        "momentum" : 0.9,
        "mo...

In [12]:
val tm = new ParameterAveragingTrainingMaster.Builder(batchSizePerWorker).
    averagingFrequency(10).
    saveUpdater(true).
    workerPrefetchNumBatches(2).
    batchSizePerWorker(batchSizePerWorker).
    repartionData(Repartition.Always).
    repartitionStrategy(RepartitionStrategy.SparkDefault).
    build()

tm: org.deeplearning4j.spark.impl.paramavg.ParameterAveragingTrainingMaster = ParameterAveragingTrainingMaster(saveUpdater=true, numWorkers=null, rddDataSetNumExamples=32, batchSizePerWorker=32, averagingFrequency=10, prefetchNumBatches=2, collectTrainingStats=false, stats=null, listeners=null, iterationCount=0, repartition=Always, repartitionStrategy=SparkDefault, storageLevel=StorageLevel(false, true, false, false, 1), storageLevelStreams=StorageLevel(false, true, false, true, 1), rddTrainingApproach=Export, exportDirectory=null, rng=java.util.Random@4f7f77f3, lastExportedRDDId=-2147483648, lastRDDExportPath=null, trainingMasterUID=1477669808919_4a31e182)

In [ ]:
val sparkNet = new SparkDl4jMultiLayer(sc, nnconf, tm)

sparkNet: org.deeplearning4j.spark.impl.multilayer.SparkDl4jMultiLayer = org.deeplearning4j.spark.impl.multilayer.SparkDl4jMultiLayer@78308d07

In [ ]:
for (i <- 0 to 10) {
    val _ = sparkNet.fit(trainData)
}